In [ ]:
# Load library imports
import sys
import torch
import random
import logging
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Point

# Load project Imports
from src.utils.config_loader import load_project_config
from src.graph_building.graph_construction import build_mesh
from src.visualisation.mapped_visualisations import plot_interactive_mesh, \
    plot_interactive_mesh_colour_coded
from src.data_ingestion.static_data_ingestion import load_land_cover_data, \
    load_elevation_data

In [ ]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

Note: CAMELS-GB is a rich data resource at the correct resolution but the time series data ends in 2015 so is insufficient for this model. Some of the static features in the dataset are still applicable:

- Land cover (e.g., % Urban, % Arable, etc.)
- Elevation and slope (from DEMs in the CAMELS dataset)
- Soil type / BFIHOST
- Hydrogeological classifications (e.g., aquifer classes)
- Geographic coordinates (for snapping)

### Build mesh ###

To align ingested data with

In [ ]:
# NB: mesh_nodes_gdf are the centroid coords, mesh_cells_gdf_polygons are polygons for e.g. averaging area
mesh_nodes_table, mesh_nodes_gdf, mesh_cells_gdf_polygons, catchment_polygon = build_mesh(
    shape_filepath=config[catchment]['paths']['gis_catchment_boundary'],
    output_path=config[catchment]['paths']['mesh_nodes_output'],
    catchment=catchment,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']
)

### Land Cover Data Ingestion ###

In [ ]:
# UKCEH LCM2023
agg_land_cover_df, catchment_gdf = load_land_cover_data(
    tif_path=config[catchment]['paths']['raw_land_cover_path'],
    csv_path=config[catchment]['paths']['land_cover_csv_path'],
    catchment=catchment,
    shape_filepath=config[catchment]['paths']['gis_catchment_boundary']
)

display(agg_land_cover_df)

**UKCEH Aggregate Class (AC)**  
    1. Broadleaf woodland  
    2. Coniferous woodland  
    3. Arable  
    4. Improved grassland  
    5. Semi-natural grassland  
    6. Mountain, heath and bog  
    7. Saltwater  
    8. Freshwater  
    9. Coastal  
    10. Built-up areas and gardens

**Mapped Classes**  
    1. Woodland  
    2. Arable  
    3. Improved grassland  
    4. Semi-natural grassland  
    5. Mountain, heath and bog  
    6. Water & Coastal  
    7. Built-up areas and gardens  

In [ ]:
logger.info(f"Preparing land cover data for visual check...")

# Convert land_cover_df to a GeoDataFrame and create a geometry column using the 'lon' and 'lat' coordinates
geometry_lc_for_plot = [Point(xy) for xy in zip(agg_land_cover_df['lon'], agg_land_cover_df['lat'])]
land_cover_gdf_for_plot = gpd.GeoDataFrame(agg_land_cover_df.copy(), geometry=geometry_lc_for_plot, crs="EPSG:4326")
    
land_cover_labels = {1: "Woodland", 2: "Arable", 3: "Improved grassland",
                     4: "Semi-Natural Grassland", 5: "Mountain, heath and bog",
                     6: "Water & Coastal", 7: "Urban and Suburban"}

land_cover_colors = {1: "#665142", 2: "#a2a182", 3: "#9CBD8B", 4: "#647D57",
                     5: "#f4ece2", 6: "#4d85ba", 7: "#8e412e"}

# Plot bounding box of data to check spatial geometry (notebook only)
land_cover_map = plot_interactive_mesh_colour_coded(
    mesh_nodes_gdf=land_cover_gdf_for_plot,
    catchment_polygon=catchment_gdf,
    map_blue=config['global']['visualisations']['maps']['map_blue'],
    esri=config['global']['visualisations']['maps']['esri'],
    esri_attr=config['global']['visualisations']['maps']['esri_attr'],
    static_output_path=f"{config[catchment]['visualisations']['maps']['static_mesh_map_output']}_land_cover",
    interactive_output_path=f"{config[catchment]['visualisations']['maps']['interactive_mesh_map_output']}_land_cover",
    category_colors=land_cover_colors,
    category_labels=land_cover_labels,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution'],
    interactive=config['global']['visualisations']['maps']['display_interactive_map']
)

# Display map in notebook
land_cover_map

### Elevation Data Ingestion ###

In [ ]:
# DIGIMAPS (via OS Terrain 5 / Terrain 50)
elevation_df = load_elevation_data(
    dir_path=config[catchment]['paths']['elevation_dir_path'],
    csv_path=config[catchment]['paths']['elevation_csv_path'],
    catchment=catchment,
    shape_filepath=config[catchment]['paths']['gis_catchment_boundary'],
    catchment_gdf=catchment_gdf,
    mesh_nodes_gdf=mesh_nodes_gdf,
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution']
)

### Slope Data Ingestion ###

In [ ]:
# DEMS / CAMELS-GB

### Soil Type Data Ingestion ###

In [ ]:
# CEH's Grid-to-Grid soil maps / HOST soil classes

### Aquifer Type Data Ingestion ###

In [ ]:
# DEFRA / British Geological Survey (BGS)

### Aquifer Transmissivity Data Ingestion ###

In [ ]:
# DEFRA / British Geological Survey (BGS)

### Geological Maps Data Ingestion ###

In [ ]:
# DIGIMAPS (BGS data via Geology Digimap)

### Distance from River (Derived) ###

In [ ]:
# Derived from DEFRA / DIGIMAP